# documentation 

generate and provide backend for services demonstrated in the documention for these notebooks



## convert notebooks to API


convert notebook to open api?


TODO: add check for readme.md in notebooks for description pages

TODO: finish export and convert other documentation to the open API format



#### the code



In [ ]:
var path = require('path')
var package = require('../package.json')
var importer = require('../Core')
var getRpcFromSpec = importer.import('get rpc from spec')
var getPermissions = importer.import('rpc permissions')
var groupPermissions = importer.import('test rpc permissions')
var getParameters = importer.import('get parameters')

// TODO: move this to patterns utility
function getTail(path, ctx) {
    var cur = path.split('.')[0]
    if(!cur) return ctx
    if(typeof ctx[cur] === 'undefined') ctx[cur] = {}
    return getTail(path.split('.').slice(1).join('.'), ctx[cur])
}

function getPathId(cell) {
    return path
        .relative('../', cell.filename)
        .replace(/\.ipynb|\s+/ig, '')
        .replace(/[^a-z0-9_\/]/ig, '')
        .replace(/\//ig, '.resources.')
        .toLowerCase()
}

function getRpcSpecFromCells(search) {
    var functions = groupPermissions(['Function', 'Available'], search)
    var resources = Object.keys(functions).reduce((obj, f) => {
        var cell = importer.interpret(f)
        var path = getPathId(cell)
        cell.params = getParameters(cell.code)
        var fn = cell.params[0]
        var tail = getTail(`${path}`, obj)
        if(typeof tail.methods === 'undefined') tail.methods = {}
        tail.methods[fn] = {
            description: cell.markdown,
            scopes: functions[f],
            id: `${package.name}.${path}.${fn}`,
            httpMethod: 'GET',
            path: `?function=${cell.questions[0]}`,
            parameters: cell.params.slice(1).reduce((o, p) => (o[p] = {
                type: 'string',
                location: 'query',
                required: true,
                description: ''
            }, o), {})
        }
        return obj
    }, {})
    return {
        title: 'Jupyter Ops',
        name: package.name,
        description: package.description,
        protocol: 'rest',
        baseUrl: process.env.FUNCTIONS_URL,
        resources: resources,
        auth: {},
        schemas: {},
        parameters: {}
    }
}

module.exports = getRpcSpecFromCells;


#### test convert notebook api?


In [6]:
var importer = require('../Core');
var getRpcSpecFromCells = importer.import('convert notebook to open api');
var getRpcFromSpec = importer.import('get rpc from spec');
var getEnvironment = importer.import('get environment');
var groupPermissions = importer.import('test get permissions');
getEnvironment('STUDY_LOCAL')

if(typeof $$ !== 'undefined') {
    $$.async()
    var spec = getRpcSpecFromCells(['study sauce.ipynb', 'rpc.ipynb'])
//    var spec = getRpcSpecFromCells('study sauce.ipynb');
    console.log(spec)
    var juypter_ops = getRpcFromSpec(spec)
    console.log(juypter_ops)
//    juypter_ops.marketing.studysauce.copyStudy({email: 'megamindbrian@gmail.com'})
    juypter_ops.core.rpc.getPermissions({search: null})
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


importing convert notebook to open api - 1 cell - documentation.ipynb[0]
compiling /Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[0] aka convert notebook to open api with module,importer,exports,__dirname,__filename,$$
Error: Line 2: Unexpected token :
    at ErrorHandler.constructError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5012:22)
    at ErrorHandler.createError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5028:27)
    at Parser.unexpectedTokenError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1985:39)
    at Parser.throwUnexpectedToken (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1995:21)
    at Parser.consumeSemicolon (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:2297:23)
    at Parser.parseExpressionStatement (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:3649:15)
    at Parser.parseStatement (/Users/briancu

Error: connect ECONNREFUSED 127.0.0.1:8010

### convert google discovery to documentation?

https://developers.google.com/apis-explorer/#s/discovery/v1/discovery.apis.getRest?api=discovery&version=v1&_h=2&

https://www.googleapis.com/discovery/v1/apis/discovery/v1/rest

https://developers.google.com/apis-explorer/#p/discovery/v1/discovery.apis.list?_h=1&




discover well-known apis?

TODO: figure out how OpenAPI, Swagger RAML do this, Google's is obvious

https://tools.ietf.org/html/rfc5785


## Export docs


### export documentation?


In [ ]:
const fs = require('fs')
const path = require('path')
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { storeAllLlamaFunctions } = importer.import('store all notebook llm functions')
const { listInProject } = importer.import('list project files');
const { askLlamaAboutCode } = importer.import('ask llm about code')
const { askLlamaToSummerize, askLlamaToGeneralize } = importer.import('ask llm to summerize')
const {cacheCells} = importer.import('cache notebook')
const {safeurl} = importer.import('domain cache tools')
const {Remarkable} = require('remarkable');
const md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

const PROJECT_PATH = path.resolve(path.join(path.dirname(__dirname), 'docs'))

async function exportAll(query) {
  // await storeAllLlamaFunctions()
  if(!fs.existsSync(PROJECT_PATH)) {
    fs.mkdirSync(PROJECT_PATH)
  }

  // TODO: get a list of all notebooks
  let notebooks = listInProject(path.resolve(__dirname, '../'), '{,*,*/,*/*/*,*/*/*/*}*.ipynb')

  let languageTemplate = importer.interpret('highlight js')
  let javadocTemplate = importer.interpret('javadoc template')

  // TODO: generate an index page that describes the index.js and __init__.py loaders

  let results = {}
  const intro = path.join(PROJECT_PATH, 'index.html')
  results[intro] = {
    output: '',
    indexOutput: '',
    title: 'Categories',
    parent: ''
  }

  let categories = []
  let namespaces = {}
  for (let i = 0; i < notebooks.length; i++) {
    // TODO: output folders and pages for every namespace and notebook
    const relative = path.relative(path.resolve(__dirname, '../'), notebooks[i])
    let category = relative.split(path.sep)[0]
    if(category.endsWith('.ipynb')) {
      category = 'Introduction'
    }

    if(!categories.includes(category)) {
      categories.push(category)
      results[intro].output += '<a href="./' +  category + '/index.html">' + category + '</a>' + '\n<br /><br />\n'
      results[intro].indexOutput += '<a href="../' +  category + '/index.html">' + category + '</a>' + '\n<br /><br />\n'
    }

    if (!fs.existsSync(path.join(PROJECT_PATH, category))) {
      fs.mkdirSync(path.join(PROJECT_PATH, category))
    }

    const catpath = path.join(PROJECT_PATH, category, 'index.html')
    if(typeof results[catpath] == 'undefined') {
      results[catpath] = {
        output: '',
        indexOutput: '',
        title: category
      }
      results[catpath].index = intro
      results[catpath].parent = '<a href="../index.html">Categories</a>'
    }


    const namespace = path.basename(notebooks[i]).replace('.ipynb', '')
    const fullpath = path.join(PROJECT_PATH, category, safeurl(namespace))
    if (!fs.existsSync(fullpath)) {
      fs.mkdirSync(fullpath)
    }

    results[catpath].output += '<a href="./' +  safeurl(namespace) + '/index.html">' + namespace + '</a>' + '\n<br /><br />\n'
    results[catpath].indexOutput += '<a href="../' +  safeurl(namespace) + '/index.html">' + namespace + '</a>' + '\n<br /><br />\n'
    let index = path.join(fullpath, 'index.html')
    results[index] = {
      output: '',
      indexOutput: '',
      title: namespace
    }
    results[index].index = catpath
    results[index].parent = '<a href="../../' +  category + '/index.html">' + category + '</a>'

    namespaces[relative] = []

    const newCache = cacheCells(notebooks[i])

    for(let j = 0; j < newCache.length; j++) {
      let cellname = newCache[j].questions[0]
      if(!cellname) {
        cellname = 'cell_' + j
      } else {
        cellname = safeurl(cellname)
      }
      let key = path.join(fullpath, cellname + '.html')
      results[key] = {
        output: '\n\n<pre class="' + newCache[j].language + '"><code>' + newCache[j].code.replaceAll('&', '&amp;').replaceAll('<', '&lt;').replaceAll('>', '&gt;') + '</code></pre>\n\n'
      }
      results[key].output += languageTemplate.code.replaceAll('${LANGUAGE}', newCache[j].language)

      let title = newCache[j].questions[0]
      if(!title) {
        title = 'Cell ' + j
      }
      results[key].index = index
      results[key].title = title
      results[key].parent = '<a href="../' +  safeurl(namespace) + '/index.html">' + namespace + '</a>'
      results[index].output += '<a href="./' +  cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'
      results[index].indexOutput += '<a href="./' +  cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'

      const description = functionCache[notebooks[i] + '[' + j + ']']
      if(!description) {
        continue
      }

      results[index].output += md.render(description.summary)
      results[key].output = md.render(description.summary) + results[key].output
      results[key].output = results[key].output + md.render(description.description)
    }

    // TODO: generate parameter, return type descriptions
    

  }


  // TODO: export all the markdown

  const paths = Object.keys(results)
  for (let i = 0; i < paths.length; i++) {
    // TODO: add navigation and full template
    let output = javadocTemplate.code
    output = output.replaceAll('${TITLE}', results[paths[i]].title)
    output = output.replaceAll('${PARENT}', results[paths[i]].parent)
    if(results[paths[i]].index)
      output = output.replaceAll('${CLASSES}', results[results[paths[i]].index].indexOutput)
    else
      output = output.replaceAll('${CLASSES}', '')
    // do this last because it should already the necessary replacements
    output = output.replaceAll('${OUTPUT}', results[paths[i]].output)
    fs.writeFileSync(paths[i], output)
  }

}

module.exports = exportAll



highlight js?

so someday these can be like mustache templates or w/e


In [ ]:
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/default.min.css">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/github-dark.css">


<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"></script>

<!-- and it's easy to individually load additional languages -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/languages/${LANGUAGE}.min.js"></script>

<script>hljs.highlightAll();</script>




javadoc template?


In [ ]:
<!DOCTYPE html>
<html>

<head>
  <meta charset='utf-8'>
  <meta http-equiv='X-UA-Compatible' content='IE=edge'>
  <title>${TITLE}</title>
  <meta name='viewport' content='width=device-width, initial-scale=1'>
  <link rel='stylesheet' type='text/css' media='screen' href='main.css'>
  <style>
    nav {
      position: fixed;
      overflow: auto;
      top: 0;
      left: 0;
      right: auto;
      bottom: 0;
      width: 200px;
    }

    header {
      background-color: #EEE;
    }

    body {
      padding-left: 200px;
    }

    @media screen and (max-width: 600px) {
      body {
        padding-left: 0;
      }

      nav {
        display: none;
      }
    }
  </style>
</head>

<body>
  <nav>
    ${PARENT}
    <br /><br />
    ${CLASSES}
  </nav>
  <header>
    ${PARENT} ${PREV} ${NEXT}
  </header>

  ${OUTPUT}
</body>

</html>